In [1]:
%cd ~/Documents/code/projects/tarsila/braz-speech-pipeline
from src.utils.database import Database
from src.clients.scp_transfer import FileTransfer

/Users/64926/Documents/code/projects/tarsila/braz-speech-pipeline


In [2]:
def query_get_segments_by_audio_id(audio_id):
    return f"""
    SELECT *
    FROM Dataset
    WHERE audio_id = {audio_id}
    """


def query_get_all_audios():
    return f"""
    SELECT *
    FROM Audio
    WHERE corpus_id is not null
    """

In [25]:
with Database() as db:
    audios_on_db = db._run_query(
        query_get_all_audios()
    )

In [7]:
with FileTransfer() as ft:
    segments_nurc = ft.read_all_files_in_folder_and_subfolders("~/BrazSpeechData/static/Dataset/data/nurc_sp")
    segments_mupe = ft.read_all_files_in_folder_and_subfolders("~/BrazSpeechData/static/Dataset/data/mupe")

<paramiko.ChannelFile from <paramiko.Channel 0 (open) window=2097152 -> <paramiko.Transport at 0x3eac2f10 (cipher aes128-ctr, 128 bits) (active; 1 open channel(s))>>>
<paramiko.ChannelFile from <paramiko.Channel 1 (open) window=2097152 -> <paramiko.Transport at 0x3eac2f10 (cipher aes128-ctr, 128 bits) (active; 1 open channel(s))>>>


In [30]:
import os
audios_on_folders = {}

for segment_path in [*segments_nurc, *segments_mupe]:
    segment_path = segment_path.replace("\\", "/")
    segment_path = segment_path.replace("\n", "")
    parts = segment_path.split("/")
    filename = os.path.basename(segment_path)
    filename, extension = os.path.splitext(filename)
    filename_path = "/".join(parts[6:])
    if extension != ".wav":
        continue

    audio_name = parts[9] if "nurc_sp" in segment_path else parts[8]
    corpus_name = 2 if parts[7] == "nurc_sp" else 1
    if audio_name in audios_on_folders:
        audios_on_folders[audio_name]["segments"].append(filename_path)
    else:
        audios_on_folders[audio_name] = {
            "corpus_name" : corpus_name,
            "segments" : [filename_path]
        }

In [48]:
analyzed_audios = []
if audios_on_db is not None and not isinstance(audios_on_db, int) and not audios_on_db.empty:
    for idx, audio in audios_on_db.iterrows():
        audio_id = audio["id"]
        audio_name = audio["name"]
        corpus_name = audio["corpus_id"]
        with Database() as db:
            segments_on_db = db._run_query(
                query_get_segments_by_audio_id(audio_id)
            )
        segments_on_db_set = set(segments_on_db["file_path"].to_list())
        found = True
        try:
            audio_on_folder = audios_on_folders[audio_name]
        except KeyError:
            found = False
            name = audio_name.split("_")[-1]
            name = " ".join(name.split(" ")[:2])
            for key in audios_on_folders.keys():
                if name in key:
                    found = True
                    audio_on_folder = audios_on_folders[key]
                    audio_name = key
                    break
        
        if not found:
            analyzed_audios.append({
                "audio_name" : audio_name,
                "corpus_name" : corpus_name,
                "count_only_on_db" : len(segments_on_db_set),
                "only_on_db" : {},
                "count_only_on_folders" : 0,
                "only_on_folders" : {}
            })
            print("Audio not found on folders:", audio_name)
            continue
        
        segments_on_folder_set = set(audios_on_folders[audio_name]["segments"])
        # Unique elements in elements_on_db and not in elements_on_folders
        only_on_db = segments_on_db_set - segments_on_folder_set

        # Unique elements in elements_on_folders and not in elements_on_db
        only_on_folders = segments_on_folder_set - segments_on_db_set

        analyzed_audios.append({
            "audio_name" : audio_name,
            "corpus_name" : corpus_name,
            "count_only_on_db" : len(only_on_db),
            "only_on_db" : only_on_db,
            "count_only_on_folders" : len(only_on_folders),
            "only_on_folders" : only_on_folders
        })
        

Audio not found on folders: PC_MA_HV242_Vicente Sacchi
Audio not found on folders: PC_MA_HV250_Severino José de Albuquerque
Audio not found on folders: PC_MA_HV251_Carlos Laporta
Audio not found on folders: PC_MA_HV252_Joaquim Timóteo Borges
Audio not found on folders: PC_MA_HV240_Vicente Eugênio Turdisi
Audio not found on folders: PC_MA_HV211_Sebastião Biano
Audio not found on folders: PC_MA_HV232_José Ferreira de Carvalho
Audio not found on folders: PC_MA_HV204_Antônio Carlos Grimaldi
Audio not found on folders: PC_MA_HV209_Zilda Noronha Miné
Audio not found on folders: PC_MA_HV207_Maria da Glória Cardia de Castro
Audio not found on folders: PC_MA_HV234_José Francisco da Silva
Audio not found on folders: PC_MA_HV203_Samuel Gonçalves
Audio not found on folders: PC_MA_HV238_Vânia Bure
Audio not found on folders: PC_MA_HV205_Cássia Navas Alves de Castro
Audio not found on folders: PC_MA_HV237_Marika Gidali
Audio not found on folders: PC_MA_HV236_Lucia Tartari Winter
Audio not found on f

In [49]:
import pandas as pd
df = pd.DataFrame(analyzed_audios)

In [50]:
df[(df.count_only_on_db > 0) |  (df.count_only_on_folders > 0)]

,audio_name,corpus_name,count_only_on_db,only_on_db,count_only_on_folders,only_on_folders
19,SP_EF_087,2,0,{},1,{data/nurc_sp/EF/SP_EF_087/audios/0082_SP_EF_0...
24,SP_EF_353,2,0,{},1,{data/nurc_sp/EF/SP_EF_353/audios/0028_SP_EF_3...
30,SP_EF_366,2,0,{},1,{data/nurc_sp/EF/SP_EF_366/audios/0286_SP_EF_3...
34,SP_EF_385,2,0,{},1,{data/nurc_sp/EF/SP_EF_385/audios/0254_SP_EF_3...
41,SP_EF_387,2,0,{},2,{data/nurc_sp/EF/SP_EF_387/audios/0335_SP_EF_3...
...,...,...,...,...,...,...
457,PC_MA_HV204_Antônio Carlos Grimaldi,1,1112,{},0,{}
458,PC_MA_HV208_Antenor Francisco,1,462,{},0,{}
459,PC_MA_HV210_Elifas Andreato,1,1183,{},0,{}
460,PC_MA_HV201_Chrisanthema Souza Santos,1,1014,{},0,{}


In [51]:
df.to_excel("audios_consistency.xlsx", index=False)